In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, History, ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from tensorflow.keras.layers import Conv2D, MaxPooling2D, concatenate, AveragePooling2D, BatchNormalization
from tensorflow.keras.layers import Input, Dense, Dropout, Activation, Flatten, ZeroPadding2D, Add
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers, regularizers
from tensorflow.keras.models import Model, load_model
#from tensorflow_addons import metrics
from google.colab import drive
from datetime import datetime
from random import shuffle
import tensorflow as tf
import numpy as np
import warnings
import os


drive.mount('/content/drive')
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

In [ ]:
def res_identity(x, filters, kern= None): 
  #renet block where dimension doesnot change.
  #The skip connection is just simple identity conncection
  #we will have 3 blocks and then input will be added

  x_skip = x # this will be used for addition with the residual block 
  f1, f2 = filters

  #first block 
  x = Conv2D(f1, kernel_size=(1, 1), strides=(1, 1), padding='valid', kernel_regularizer=kern)(x)
  x = BatchNormalization()(x)
  x = Activation('relu')(x)

  #second block # bottleneck (but size kept same with padding)
  x = Conv2D(f1, kernel_size=(3, 3), strides=(1, 1), padding='same', kernel_regularizer=kern)(x)
  x = BatchNormalization()(x)
  x = Dropout(0.4)(x)
  x = Activation('relu')(x)

  # third block activation used after adding the input
  x = Conv2D(f2, kernel_size=(1, 1), strides=(1, 1), padding='valid', kernel_regularizer=kern)(x)
  x = BatchNormalization()(x)
  x = Dropout(0.4)(x)
  x = Activation('relu')(x)

  # add the input 
  x = Add()([x, x_skip])
  x = Activation('relu')(x)

  return x

In [ ]:
def res_conv(x, s, filters, kern= None):
  '''
  here the input size changes''' 
  x_skip = x
  f1, f2 = filters

  # first block
  x = Conv2D(f1, kernel_size=(1, 1), strides=(s, s), padding='valid', kernel_regularizer=kern)(x)
  # when s = 2 then it is like downsizing the feature map
  x = BatchNormalization()(x)
  x = Activation('relu')(x)

  # second block
  x = Conv2D(f1, kernel_size=(3, 3), strides=(1, 1), padding='same', kernel_regularizer=kern)(x)
  x = BatchNormalization()(x)
  x = Activation('relu')(x)

  #third block
  x = Conv2D(f2, kernel_size=(1, 1), strides=(1, 1), padding='valid', kernel_regularizer=kern)(x)
  x = BatchNormalization()(x)
  x = Dropout(0.4)(x)
  x = Activation('relu')(x)

  # shortcut 
  x_skip = Conv2D(f2, kernel_size=(1, 1), strides=(s, s), padding='valid', kernel_regularizer=kern)(x_skip)
  x_skip = BatchNormalization()(x_skip)
  x = Dropout(0.4)(x)
  x_skip = Activation('relu')(x_skip)

  # add 
  x = Add()([x, x_skip])
  x = Activation('relu')(x)

  return x

def channel_attention(f, filtros: int, ratio: int= 1, nombre: str= None):
  reduccion= int(filtros // ratio)
  Favr= tf.reduce_mean(f, axis= [-1], keepdims= True)
  Fmax= tf.reduce_max(f, axis=[-1], keepdims= True)

  Davr_r= Dense(reduccion, activation= 'relu')(Favr)
  Dmax_r= Dense(reduccion, activation= 'relu')(Fmax)

  Davr_a= Dense(filtros, activation= 'relu')(Davr_r)
  Dmax_a= Dense(filtros, activation= 'relu')(Dmax_r)
  
  return Activation('sigmoid', name= nombre)(tf.add(Davr_a, Dmax_a))


def spatial_attention(f, kernel_s: int= 7, nombre: str= None):
  Favr= tf.reduce_mean(f, axis= [-1], keepdims= True)
  Fmax= tf.reduce_max(f, axis= [-1], keepdims= True)

  C_am= tf.concat([Favr, Fmax], axis= -1)

  Ms= Conv2D(1, kernel_size= kernel_s, padding= 'same')(C_am)
  Ms= BatchNormalization()(Ms)

  return Activation('sigmoid', name= nombre)(Ms)


def CBAM(f, nombre: str= None):
  f_1= channel_attention(f, f.shape[-1], 16)
  r_1= tf.math.multiply(f, f_1)
  
  f_2= spatial_attention(r_1)
  r_2= tf.math.multiply(r_1, f_2)

  return tf.add(f, r_2, name= nombre)

In [ ]:
input_im = Input(shape=(480, 480, 3)) # cifar 10 images size
x = ZeroPadding2D(padding=(3, 3))(input_im)

  # 1st stage
  # here we perform maxpooling, see the figure above

x = Conv2D(64, kernel_size=(7, 7), strides=(2, 2))(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPooling2D((2, 2), strides=(2, 2))(x)

  #2nd stage 
  # frm here on only conv block and identity block, no pooling

x = res_conv(x, s=1, filters=(64, 256))
x= CBAM(x)
x = res_identity(x, filters=(64, 256))
x= CBAM(x)
x = res_identity(x, filters=(64, 256))
x= CBAM(x)
  # 3rd stage

x = res_conv(x, s=2, filters=(128, 512))
x= CBAM(x)
x = res_identity(x, filters=(128, 512))
x= CBAM(x)
x = res_identity(x, filters=(128, 512))
x= CBAM(x)
x = res_identity(x, filters=(128, 512))

  # 4th stage

x = res_conv(x, s=2, filters=(256, 1024), kern= tf.keras.regularizers.L2(0.01))
x= CBAM(x)
x = res_identity(x, filters=(256, 1024), kern= tf.keras.regularizers.L2(0.01))
x= CBAM(x)
x = res_identity(x, filters=(256, 1024), kern= tf.keras.regularizers.L2(0.01))
x= CBAM(x)
x = res_identity(x, filters=(256, 1024), kern= tf.keras.regularizers.L2(0.01))
x= CBAM(x)
x = res_identity(x, filters=(256, 1024), kern= tf.keras.regularizers.L2(0.01))
x= CBAM(x)
x = res_identity(x, filters=(256, 1024), kern= tf.keras.regularizers.L2(0.01))
x= CBAM(x)


  # 5th stage

x = res_conv(x, s=2, filters=(512, 2048), kern= tf.keras.regularizers.L2(0.01))
x= CBAM(x)
x = res_identity(x, filters=(512, 2048), kern= tf.keras.regularizers.L2(0.01))
x= CBAM(x)
x = res_identity(x, filters=(512, 2048), kern= tf.keras.regularizers.L2(0.01))
x= CBAM(x)

  # ends with average pooling and dense connection

x = AveragePooling2D((2, 2), padding='same')(x)

x = Flatten()(x)
#x = Dense(4096, activation= "relu")(x)
x = Dropout(0.6)(x)
x = Dense(1, activation='sigmoid')(x)#, kernel_initializer='he_normal')(x) #multi-class

  # define the model 

model = Model(inputs=input_im, outputs=x, name='Resnet50')

#model.summary()

#tf.keras.models.save_model(model, "/content/drive/MyDrive/Proyectos/Sharon/Modelos guardados/ResNet50/ResNet50.hdf5")

In [ ]:
#!unzip "/content/drive/MyDrive/Sharon/Base de datos/Base_Procesada_Mecidor/Val/Val.zip" -d "/content/sample_data/"
#!unzip "/content/drive/MyDrive/Sharon/Base de datos/Base_Procesada_Mecidor/Ent/Ent.zip" -d "/content/sample_data/"

#Kaggle
!unzip "/content/drive/MyDrive/Sharon/Base de datos/Kaggle/Val.zip" -d "/content/sample_data/"
!unzip "/content/drive/MyDrive/Sharon/Base de datos/Kaggle/Ent.zip" -d "/content/sample_data/"

In [ ]:
input_dir= "/content/sample_data/Ent"
target_dir= "/content/sample_data/Val"

generador_ent= ImageDataGenerator(rescale= 1/255.0)
generador_val= ImageDataGenerator(rescale= 1/255.0)


ent_gen= generador_ent.flow_from_directory(input_dir,
                                           class_mode= 'binary',
                                           target_size=(480, 480),
                                           batch_size= 1,
                                           color_mode= "rgb",
                                           shuffle= True,
                                           )

val_gen= generador_val.flow_from_directory(target_dir,
                                           class_mode= 'binary',
                                           target_size=(480, 480),
                                           color_mode= "rgb",
                                           batch_size= 1,
                                           shuffle= True,
                                           )

In [ ]:
def entrenamient_modelo(train_gen, val_gen, DIR_save, epocas= 30):

    #class_weights = {0:1., 1:3.}

    #Generamos los callback necesarios
    checkpoint= ModelCheckpoint(DIR_save+ "09122021_CBAM.hdf5", monitor= 'val_loss', save_best_only= True, mode= 'min', save_weights_only= False)
    reduceLROnPlat= ReduceLROnPlateau(monitor= 'val_loss', factor= 0.8, patience= 15, min_delta= 0.001, cooldown= 10, min_lr= 0.00001)
    tensorboard_callback = TensorBoard(log_dir= datetime.now().strftime("%Y%m%d-%H%M%S"))
    callbacks= [EarlyStopping(patience= 30,  monitor='val_loss'), checkpoint, tensorboard_callback, reduceLROnPlat]
    metrica= [tf.keras.metrics.AUC(),
              tf.keras.metrics.BinaryAccuracy(),
              tf.keras.metrics.Recall(),
              tf.keras.metrics.Precision()]

    #Compilamos con la funcion perdida y el optimizador que utilizaremos
    #model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate= 0.0001), loss = 'binary_crossentropy', metrics= metrica)
    history= model.fit(train_gen, batch_size= 1, epochs=epocas, validation_data=val_gen, callbacks=callbacks, initial_epoch= 6)#, class_weight= class_weights)

def carga_mod(DIR):
    return tf.keras.models.load_model(DIR)

In [ ]:
DIR_save_best= "/content/drive/MyDrive/Sharon/ModeloGuardado/Kaggle/"
DIR_load_best= '/content/drive/MyDrive/Sharon/ModeloGuardado/Kaggle/08122021_CBAM.hdf5'

model= carga_mod(DIR_load_best)
entrenamient_modelo(ent_gen, val_gen, DIR_save_best, 300)

### Segmentación

In [ ]:
from tensorflow.keras.layers import Conv2D, BatchNormalization, Conv2DTranspose, MaxPooling2D, UpSampling2D, concatenate, Dropout
from tensorflow.keras.callbacks import EarlyStopping, History, ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Model, load_model
import matplotlib.pyplot as plt
from datetime import datetime
from google.colab import drive
import os
import tensorflow as tf
import numpy as np
import warnings


drive.mount('/content/drive')
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [ ]:
# Dirección del modelo y carga del modelo
model_path= '/content/drive/MyDrive/Sharon/ModeloGuardado/Kaggle/09122021_CBAM.hdf5'

model_2= tf.keras.models.load_model(model_path)
#model_2.summary()

In [ ]:
input_im = Input(shape=(160, 160, 3)) # cifar 10 images size
x = ZeroPadding2D(padding=(3, 3))(input_im)

  # 1st stage
  # here we perform maxpooling, see the figure above

x = Conv2D(64, kernel_size=(7, 7), strides=(2, 2))(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPooling2D((2, 2), strides=(2, 2))(x)

  #2nd stage 
  # frm here on only conv block and identity block, no pooling

x = res_conv(x, s=1, filters=(64, 256))
x= CBAM(x)
x = res_identity(x, filters=(64, 256))
x= CBAM(x)
x = res_identity(x, filters=(64, 256))
x= CBAM(x)
  # 3rd stage

x = res_conv(x, s=2, filters=(128, 512))
x= CBAM(x)
x = res_identity(x, filters=(128, 512))
x= CBAM(x)
x = res_identity(x, filters=(128, 512))
x= CBAM(x)
x = res_identity(x, filters=(128, 512))

  # 4th stage

x = res_conv(x, s=2, filters=(256, 1024), kern= tf.keras.regularizers.L2(0.01))
x= CBAM(x)
x = res_identity(x, filters=(256, 1024), kern= tf.keras.regularizers.L2(0.01))
x= CBAM(x)
x = res_identity(x, filters=(256, 1024), kern= tf.keras.regularizers.L2(0.01))
x= CBAM(x)
x = res_identity(x, filters=(256, 1024), kern= tf.keras.regularizers.L2(0.01))
x= CBAM(x)
x = res_identity(x, filters=(256, 1024), kern= tf.keras.regularizers.L2(0.01))
x= CBAM(x)
x = res_identity(x, filters=(256, 1024), kern= tf.keras.regularizers.L2(0.01))
x= CBAM(x)

# 5th stage

x = res_conv(x, s=2, filters=(512, 2048), kern= tf.keras.regularizers.L2(0.01))
x= CBAM(x)
x = res_identity(x, filters=(512, 2048), kern= tf.keras.regularizers.L2(0.01))
x= CBAM(x)
x = res_identity(x, filters=(512, 2048), kern= tf.keras.regularizers.L2(0.01))
x= CBAM(x)

  # ends with average pooling and dense connection

#x = AveragePooling2D((2, 2), padding='same')(x)

model= Model(inputs= input_im, outputs= x)
#model.summary()

In [ ]:
for i, capa in enumerate(model_2.layers):
  #print(i)
  #print(capa)
  #print('-----------------------------------')
  if i < 481:
    model.layers[i].set_weights= capa.weights

In [ ]:
def congelar_capas(model1):
  for layer in model1.layers:
    layer.trainable= True
  return model1

def cop_crong(modelo_ant, dir_a):
  x= model.get_layer(dir_a)
  return x.output

def up_conc(capa, res):
  x= UpSampling2D(2)(capa)
  return concatenate([x, res])

def conv_f(ker, x, nm):
  x= Conv2D(ker, 3, padding= "same", activation= "relu", name= 'conv2d_'+ str(nm))(x)
  x= BatchNormalization(name= 'batch_normalization_'+ str(nm))(x)
  x= Conv2D(ker, 3, padding= "same", activation= "relu", name= 'conv2d_'+ str(nm+1))(x)
  x= BatchNormalization(name= 'batch_normalization_'+ str(nm+1))(x)
  return Conv2DTranspose(int(ker/2), 3, padding= "same")(x)

def conv_v(ker, x, nm):
  x= Conv2D(ker, 3, padding= "same", activation= "relu", name= 'conv2d_'+ str(nm))(x)
  x= BatchNormalization(name= 'batch_normalization_'+ str(nm))(x)
  x= Conv2D(ker, 3, padding= "same", activation= "relu", name= 'conv2d_'+ str(nm+1))(x)
  x= BatchNormalization(name= 'batch_normalization_'+ str(nm+1))(x)
  x= Conv2D(ker, 3, padding= "same", activation= "relu", name= 'conv2d_'+ str(nm+2))(x)
  x= BatchNormalization(name= 'batch_normalization_'+ str(nm+2))(x)
  return Conv2DTranspose(int(ker/2), 3, padding= "same")(x)

def salida_d(ker, x, nm):
  x= Conv2D(ker, 3, padding= "same", activation= "relu", name= 'conv2d_'+ str(nm))(x)
  x= BatchNormalization(name= 'batch_normalization_'+ str(nm))(x)
  x= Conv2D(ker/2, 3, padding= "same", activation= "relu", name= 'conv2d_'+ str(nm+1))(x)
  x= BatchNormalization(name= 'batch_normalization_'+ str(nm+1))(x)
  return Conv2D(2, 3, padding= "same", activation= "softmax", name= 'conv2d_'+ str(nm+2))(x)

In [ ]:
#model_1= Model(inputs= model.inputs, outputs= model.layers[33].output)
model= congelar_capas(model)

x1= cop_crong(model, 'activation')
x2= cop_crong(model, 'tf.math.add_35')
x3= cop_crong(model, 'activation_42')
x4= cop_crong(model, 'tf.math.add_53')
x5= cop_crong(model, 'tf.math.add_59')

x5= conv_v(1024, x5, 227)
x6= up_conc(x5, x4)


x7= conv_v(1024, x6, 240)
x8= up_conc(x7, x3)

x9= conv_f(512, x8, 289)
x10= up_conc(x9, x2)


x11= conv_f(256, x10, 298)
x12= up_conc(x11, x1)

x13= conv_f(128, x12, 300)
x14= UpSampling2D(2)(x13)

x15= salida_d(64, x14, 310)


new_model= Model(inputs= model.input, outputs= x15)

#new_model.summary()

In [ ]:
!unzip "/content/drive/MyDrive/Sharon/Base de datos/Segmentacion/Hem/Val.zip" -d "/content/sample_data/"
!unzip "/content/drive/MyDrive/Sharon/Base de datos/Segmentacion/Hem/Ent.zip" -d "/content/sample_data/"

In [ ]:
ent_input_dir= "/content/sample_data/Ent/org"
ent_target_dir= "/content/sample_data/Ent/mask"

val_input_dir= "/content/sample_data/Val/org"
val_target_dir= "/content/sample_data/Val/mask"

input_img_paths = sorted(
    [
        os.path.join(ent_input_dir, fname)
        for fname in os.listdir(ent_input_dir) 
        if fname.endswith(".png")
    ]
)

target_img_paths = sorted(
    [
        os.path.join(ent_target_dir, fname)
        for fname in os.listdir(ent_target_dir)
        if fname.endswith(".png") and not fname.startswith(".")
    ]
)

val_input_paths = sorted(
    [
        os.path.join(val_input_dir, fname)
        for fname in os.listdir(val_input_dir)
        if fname.endswith(".png") and not fname.startswith(".")
    ]
)

val_target_paths = sorted(
    [
        os.path.join(val_target_dir, fname)
        for fname in os.listdir(val_target_dir)
        if fname.endswith(".png") and not fname.startswith(".")
    ]
)

ID_input_t= np.arange(len(input_img_paths))
ID_input_v= np.arange(len(val_input_paths))

print(len(input_img_paths))
print(len(target_img_paths))
print(len(val_target_paths))
print(len(val_input_paths))

In [ ]:
#Generador de los ejemplos de entrada
class MiClasificacion(tf.keras.utils.Sequence):
    def __init__(self, input_img_paths, target_img_paths, ID_input, batch_size= 32, img_size= (304, 304, 3), train= True):

        self.batch_size = batch_size
        self.img_size = img_size
        self.input_img_paths = input_img_paths
        self.target_img_paths = target_img_paths
        self.ID_input = ID_input
        self.cont= 0

    def __len__(self):
        # Calcula el numero de pasos por epoca.
        return len(self.ID_input) // self.batch_size

    def __getitem__(self, idx):
        """Returns tuple (input, target) correspond to batch #idx."""

        # Combinamos los datos en cada epoca, pero solo para los datos de entrenamiento
        if (self.cont== len(self.ID_input) // self.batch_size) or (self.cont== 0):
            np.random.shuffle(self.ID_input)
            self.cont= 0
            
        i = idx * self.batch_size
        batch_img= []
        batch_tar= []
        self.cont += 1

        #De acuerdo a los indices en ID_input tomamos las imagenes de los paths correspondientes
        for ig in self.ID_input[i : i + self.batch_size]:
            batch_img.append(self.input_img_paths[ig])
            batch_tar.append(self.target_img_paths[ig])

        X, Y= self.__data_generation(batch_img, batch_tar)                                          # Mandamos llamar a la funcion generadora de los ejemplos
        
        
        #print(X.shape)
        #print(Y.shape)
        return X, Y

    def __data_generation(self, biip, btip):
            
        # Creacion del tensor con dimensiones las dimensiones de entrada
        x = np.zeros((self.batch_size,) + self.img_size + (3,), dtype="float32")                   # Generamos un tensor que almacenara las imagenes importadas     
        for j, path in enumerate(biip):
            img = load_img(path, target_size=self.img_size)                                         # Cragamos las imagenes de entrada 
            x[j] = np.array(img)/255.0                                                              # Normalizamos y agregamos la imagen al tensor

        y = np.zeros((self.batch_size,) + self.img_size + (1,), dtype="uint8")                      # Generamos un tensor que almacenara las mascaras importadas 
        for j, path in enumerate(btip):
            img = load_img(path,target_size= self.img_size,color_mode="grayscale")                  # Importamos la imagne en escala de grises
            y[j] = np.expand_dims(img, 2)
            for r in range(y.shape[1]):
                for g in range(y.shape[2]):
                    if y[j, r, g, 0]!= 0:                                                           # Nos aseguramos de que las venas etiquetadas resulten se un 1
                        y[j, r, g, 0]= 1

        return x, y

In [ ]:
def entrenamient_modelo(train_gen, val_gen, DIR_save, epocas= 30):

    #Generamos los callback necesarios
    checkpoint= ModelCheckpoint(DIR_save+ "12122021_H_CBAM.hdf5", monitor= 'val_loss', save_best_only= True, mode= 'min', save_weights_only= False)
    #reduceLROnPlat= ReduceLROnPlateau(monitor= 'val_loss', factor= 0.8, patience= 5, min_delta= 0.001, cooldown= 10, min_lr= 0.0001)
    #tensorboard_callback = TensorBoard(log_dir= datetime.now().strftime("%Y%m%d-%H%M%S"))
    callbacks= [EarlyStopping(patience= 12,  monitor='val_loss'), History(), checkpoint]#, reduceLROnPlat, tensorboard_callback]

    #Compilamos con la funcion perdida y el optimizador que utilizaremos
    new_model.compile(optimizer="Adam", loss= "sparse_categorical_crossentropy", metrics= ['accuracy']) #loss= "binary_crossentropy", metrics= ['accuracy'])
    history= new_model.fit(train_gen, epochs=epocas, validation_data=val_gen, callbacks=callbacks)

def carga_mod(DIR):
    return tf.keras.models.load_model(DIR)

img_size = (160, 160)
num_classes = 2; batch_size_t = 16; batch_size_v = 16

#Creamos los generadoras para el entrenamiento y la validación
train_gen = MiClasificacion(
    input_img_paths, target_img_paths, ID_input_t, batch_size= batch_size_t, img_size= img_size
)

val_gen = MiClasificacion(
    val_input_paths, val_target_paths, ID_input_v, batch_size=batch_size_v, img_size= img_size
)


DIR_save_best= '/content/drive/MyDrive/Sharon/ModeloGuardado/Segmentacion/'
#DIR_LOAD= '/content/drive/MyDrive/Sharon/ModeloGuardado/Segmentacion/09122021_H_CBAM.hdf5'

#new_model= carga_mod(DIR_LOAD)
entrenamient_modelo(train_gen, val_gen, DIR_save_best, 10)

#Resultados de la segmentacion

In [ ]:
from tensorflow.keras.preprocessing.image import load_img
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from google.colab import drive
from PIL import ImageOps
import tensorflow as tf
import numpy as np
import cv2 as cv
#from IPython.display import Image, display
import PIL


drive.mount('/content/drive')

In [ ]:
DIR= '/content/drive/MyDrive/Sharon/ModeloGuardado/Segmentacion/12122021_H_CBAM.hdf5'
model= tf.keras.models.load_model(DIR)

In [ ]:
def cambio_tam(img, msk):
  gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

  gray = cv.GaussianBlur(gray, (7, 7), 3)
  t, dst = cv.threshold(gray, 0, 255, cv.THRESH_BINARY | cv.THRESH_TRIANGLE)
  contours, _= cv.findContours(dst, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
  img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
  a = []

  for c in contours:
    area = cv.contourArea(c)
    a.append(area)
        
  for f in contours:
    area = cv.contourArea(f)
    if area == max(a):
        x, y, w, h = cv.boundingRect(f)
        imgr = img[y:y + h, x:x + w]
        mask = msk[y:y + h, x:x + w]

  return imgr, mask

def prediccion_m(model, img, mostrar= False, h=0):
  img= (np.array(img).reshape(-1,160,160,3))/255.0
  val_preds = model.predict(img)
  mask = np.argmax(val_preds[0], axis=-1)
  mask = np.expand_dims(mask, axis=-1)
  img = PIL.ImageOps.autocontrast(tf.keras.preprocessing.image.array_to_img(mask))
  if mostrar== True:
    plt.imshow(img, cmap="gray")
    plt.show()
  return img

def equalize_clahe_color(img):
    cla = cv.createCLAHE(clipLimit=1.5)     
    channels = cv.split(img)     
    eq_channels = []
    for ch in channels:         
        eq_channels.append(cla.apply(ch))     
        eq_image = cv.merge(eq_channels)     
    return eq_image

def pre_procesamiento(img):
    return cv.cvtColor(equalize_clahe_color(cv.cvtColor(img, cv.COLOR_BGR2RGB)), cv.COLOR_BGR2RGB)

def prediccion_m(model, img):
  img= (np.array(img).reshape(-1,160,160,3))/255.0
  val_preds = model.predict(img)
  mask = np.argmax(val_preds[0], axis=-1)
  mask = np.expand_dims(mask, axis=-1)
  img = PIL.ImageOps.autocontrast(tf.keras.preprocessing.image.array_to_img(mask))
  
  return np.array(img)

# Genera los patches de entrada para el modelo
def generar_patch(img, tam= 160):
    f= []; g= 0; c= 0
    for t in range(int(img.shape[0]/tam)):
        for j in range(int(img.shape[1]/tam)):
            f.append(img[g:g+ tam, c: c+ tam, :])
            c+= tam
        if img.shape[1]% tam != 0:
            f.append(cv.resize(img[g:g+ tam, c: c+ (img.shape[1]-c), :], (tam, tam), interpolation = cv.INTER_AREA))
        g+= tam; c= 0
        
    for t in range(int(img.shape[1]/tam)):
        f.append(cv.resize(img[g:g+ (img.shape[0]- g), c: c+ tam, :], (tam, tam), interpolation = cv.INTER_AREA))
        c+= tam
    if img.shape[1]% tam != 0:
        f.append(cv.resize(img[g:g+ (img.shape[0]-g), c: c+ (img.shape[1]-c), :], (tam,tam), interpolation = cv.INTER_AREA))
    
    return f

def construccion_img(img, model, f, n= 0, c= 0, g= 0, tam= 160):
    img_pred= np.zeros((img.shape[0], img.shape[1]))
    for t in range(int(img.shape[0]/tam)):
        for j in range(int(img.shape[1]/tam)):
            p= prediccion_m(model, f[n])
            img_pred[g: g+ tam, c: c+ tam]= p
            c+= tam; n+= 1
        if img.shape[1]% tam != 0:
            p= cv.resize(prediccion_m(model, f[n]), (img.shape[1]-c, tam), interpolation = cv.INTER_AREA)
            img_pred[g: g+ tam, c:]= p
        g+= tam; c= 0; n+= 1
        
    for t in range(int(img.shape[1]/tam)):
        p= cv.resize(prediccion_m(model, f[n]), (tam, img.shape[0]-g), interpolation = cv.INTER_AREA)
        img_pred[g:, c: c+ tam]= p
        c+= tam; n+= 1
    if img.shape[1]% tam != 0:
        p= cv.resize(prediccion_m(model, f[n]), (img.shape[1]-c, img.shape[0]-g), interpolation = cv.INTER_AREA)
        img_pred[g:, c:]= p
    
    return img_pred

def imagenes_comp(img, mask, s1= None, s2= None, s3= None, mostrar= False, guardar= False):
  img_t= np.zeros((img.shape[0], img.shape[1], 3), dtype= np.float)
  conv= np.zeros((img.shape[0], img.shape[1], 3), dtype= np.float)
  img_t[:, :, 1]= img

  conv[:,:,0]= mask[:,:,0]
  conv[:,:,1]= img

  if mostrar== True:
    plt.imshow(img_t, cmap= 'gray')
    plt.show()

    plt.imshow(mask, cmap= 'gray')
    plt.show()

    plt.imshow(conv)
    plt.show()
  
  if guardar== True:
    plt.imsave(s1, img_t/255.0)
    plt.imsave(s2, mask/255.0)
    plt.imsave(s3, conv/255.0)


def prediccion_final(model, DIR, DIR_M):
  img= cv.imread(DIR)
  mask= tf.keras.preprocessing.image.load_img(DIR_M)
  mask= tf.keras.preprocessing.image.img_to_array(mask)

  img, msk= cambio_tam(img, mask)
  img= equalize_clahe_color(img)

  f= generar_patch(img)
  pred= construccion_img(img, model, f)

  return pred, msk

In [ ]:
DIR_M= "/content/drive/MyDrive/Sharon/Base de datos/Pruebas/Hemorragias/mask/IDRiD_17_HE.tif"
DIR= "/content/drive/MyDrive/Sharon/Base de datos/Pruebas/Hemorragias/org/IDRiD_17.jpg"

s1= "/content/drive/MyDrive/Sharon/Base de datos/Pruebas/Hemorragias/com/p_res_cbam.png"
s2= "/content/drive/MyDrive/Sharon/Base de datos/Pruebas/Hemorragias/com/m_res_cbam.png"
s3= "/content/drive/MyDrive/Sharon/Base de datos/Pruebas/Hemorragias/com/conv_res_cbam.png"

pred, mask= prediccion_final(model, DIR, DIR_M)

imagenes_comp(pred, mask, s1, s2, s3, True, True)